# Optimization
The linear score function and its SVM loss function we developed formulated as:

$$f(x_i, W) =  W x_i$$
$$L = \frac{1}{N} \sum_i \sum_{j\neq y_i} \left[ \max(0, f(x_i; W)_j - f(x_i; W)_{y_i} + 1) \right] + \alpha R(W)$$
$$R(W) = \sum_k\sum_l W_{k,l}^2$$

A setting of the parameters \\(W\\) that produced predictions for examples \\(x_i\\) consistent with their ground truth labels \\(y_i\\) would also have a very low loss \\(L\\). We are now going to introduce the third and last key component: **optimization**. Optimization is the process of finding the set of parameters \\(W\\) that minimize the loss function.

## Random search
The first (very bad) idea that may come to mind is to simply try out many different random weights and keep track of what works best. This procedure might look as follows:

In [ ]:
import numpy as np

# assume X_train is the data where each column is an example (e.g. 3073 x 50,000)
# assume Y_train are the labels (e.g. 1D array of 50,000)
# assume the function L evaluates the loss function

bestloss = float("inf")
for num in range(1000):
    W = np.random.randn(10, 3073)
    loss = L(X_train, Y_train, W)
    if loss < bestloss:
        bestloss = loss
        bestW = W
    print('in attempt %d the loss was %f, best %f' % (num, loss, bestloss))


In [ ]:
# Assume X_test is [3073 x 10000], Y_test [10000 x 1]

scores = Wbest.dot(X_test)
Y_predict = np.argmax(scores, axis = 0)
np.mean(Y_predict == Y_test)

## Random Local Search

We will start out with a random \\(W\\), generate random perturbations \\(\delta W\\) to it and if the loss at the perturbed \\(W + \delta W\\) is lower, we will perform an update. The code for this procedure is as follows:

In [ ]:
W = np.random.randn(10, 3073) * 0.001
bestLoss = float("inf")
for i in range(1000):
    step_size = 0.0001
    Wtry = W + np.random.randn(10, 3073) * step_size
    loss = L(X_train, Y_train, Wtry)
    if loss < bestLoss:
        W = Wtry
        bestLoss = loss
    print('iter %d loss is %f' % (i, bestLoss))

## Gradient Descent

We can compute the best direction along which we should change our weight vector that is mathematically guaranteed to be the direction of the steepest descend in the weight-space. This direction will be related to the gradient of the loss function.

### Slope and Gradient
In one-dimensional functions, the slope is the instantaneous rate of change of the function at any point you might be interested in.

$$\frac{df(x)}{dx} = \lim_{h\ \to 0} \frac{f(x + h) - f(x)}{h}$$

The gradient is just a vector of slopes (more commonly referred to as **derivatives**) for each dimension in the input space. When the functions of interest take a vector of numbers instead of a single number, we call the derivatives **partial derivatives**, and the gradient is simply the vector of partial derivatives in each dimension.

### Numerical Gradient

Following the gradient formula we gave above, the code below iterates over all dimensions one by one, makes a small change **h** along that dimension and calculates the partial derivative of the loss function along that dimension by seeing how much the function changed. The variable **grad** holds the full gradient in the end.

In [ ]:
def eval_numerical_gradient(f, x):
    fx = f(x)
    grad = np.zeros(x.shape)
    h = 0.00001
    
    it = np.nditer(x, flags=['multi_index'], op_flags=['readwrite'])
    while not it.finished:
        ix = it.multi_index
        old_value = x[ix]
        x[ix] = old_value + h
        fxh = f(x)
        x[ix] = old_value
        
        grad[ix] = (fxh - fx) / h
        it.iternext()
        
    return grad

### Analytic Gradient

Lets use the example of the SVM loss function for a single datapoint:

$$L_i = \sum_{j\neq y_i} \left[ \max(0, w_j^Tx_i - w_{y_i}^Tx_i + \Delta) \right]$$

Taking the gradient with respect to \\(w_{y_i}\\)

$$\nabla_{w_{y_i}} L_i = - \left( \sum_{j\neq y_i} \mathbb{1}(w_j^Tx_i - w_{y_i}^Tx_i + \Delta > 0) \right) x_i$$

Notice that this is the gradient only with respect to the row of \\(W\\) that corresponds to the correct class. For the other rows where \\(j \neq y_i\\) the gradient is:

$$\nabla_{w_j} L_i = \mathbb{1}(w_j^Tx_i - w_{y_i}^Tx_i + \Delta > 0) x_i$$

The gradient for regularization is:
$$\nabla_W R(W) = \lambda W$$

So, the whole formula of gradient for the SVM loss function is:

$$\nabla_W L(W) = \frac{1}{N} \sum_{i} \nabla_{w_{j}} L_i + \lambda W$$

Once you derive the expression for the gradient it is straight-forward to implement the expressions and use them to perform the gradient update.

In [ ]:
# Vanilla Gradient Descent

while True:
    weights_grad = evaluate_gradient(loss_func, data, weights)
    weights += (-step_size) * weights_grad

In [ ]:
# Vanilla Minibatch Gradient Descent

while True:
    data_batch = sample_training_data(data, 256)
    weights_grad = evaluate_gradient(loss_func, data_batch, weights)
    weights += (-step_size) * weights_grad